In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/데이터마이닝/DATA/건강검진정보_학습데이터.xlsx')
df

,성별,신장,체중,허리둘레,수축기혈압,이완기혈압,식전혈당,트리글리세라이드,콜레스테롤,높은혈압,높은혈당,높은중성지방,낮은콜레스테롤,복부비만,BMI,대사증후군수,대사증후군판별,비만도판별
0,남,170,70,82.0,124,80,83,95,57,0,0,0,0,0,24.22,0,정상,정상
1,여,135,40,81.0,118,63,112,168,42,0,1,1,1,0,21.95,3,위험군,정상
2,여,145,50,78.0,148,67,98,70,57,1,0,0,0,0,23.78,1,주의군,정상
3,여,150,65,83.0,118,68,111,106,66,0,1,0,0,0,28.89,1,주의군,과체중
4,여,155,55,74.0,110,77,98,193,35,0,0,1,1,0,22.89,2,주의군,정상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439,여,150,50,69.0,100,64,93,307,37,0,0,1,1,0,22.22,2,주의군,정상
440,여,155,50,78.0,124,70,120,85,62,0,1,0,0,0,20.81,1,주의군,정상
441,여,155,55,77.0,100,71,88,56,71,0,0,0,0,0,22.89,0,정상,정상
442,남,160,60,80.2,150,90,104,88,71,1,1,0,0,0,23.44,2,주의군,정상


In [ ]:
df.columns

Index(['성별', '신장', '체중', '허리둘레', '수축기혈압', '이완기혈압', '식전혈당', '트리글리세라이드', '콜레스테롤',
       '높은혈압', '높은혈당', '높은중성지방', '낮은콜레스테롤', '복부비만', 'BMI', '대사증후군수', '대사증후군판별',
       '비만도판별'],
      dtype='object')

지도학습

In [ ]:
# 필요한 열 추출
df = df[['신장','체중','비만도판별']]
df.head()

,신장,체중,비만도판별
0,170,70,정상
1,135,40,정상
2,145,50,정상
3,150,65,과체중
4,155,55,정상


In [ ]:
# 데이터 변환 : 타겟 (비만도판별 예측), 범주형 데이터를 수치형 데이터로

# scikit-learn 라이브러리
from sklearn.preprocessing import LabelEncoder

# 데이터 변환한 데이터를 저장하기 위한 데이터프레임 형성
dft = df.copy()
# 반드시 알아야함 - df.copy()
# 원 데이터프레임 보존

# OrdinalEncoder 객체 생성
label_encoder = LabelEncoder()

# '비만도판별' 열의 범주형 데이터를 수치형 데이터로 변환
dft['비만도판별'] = label_encoder.fit_transform(dft['비만도판별'])
dft.head()

,신장,체중,비만도판별
0,170,70,3
1,135,40,3
2,145,50,3
3,150,65,0
4,155,55,3


In [ ]:
# test 셋과 train 셋을 나누기
from sklearn.model_selection import train_test_split
feature = dft.loc[:, ['신장', '체중']]
target = dft['비만도판별']

X_train, X_test, y_train, y_test = train_test_split(feature, target, shuffle=True, stratify = target, random_state=156)

In [ ]:
# 모델링 및 평가
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import classification_report



In [ ]:
# 평가지표 함수
def get_clf_eval(y_test, pred):
  accuracy = accuracy_score(y_test, pred)
  precision = precision_score(y_test, pred,average='macro')
  recall = recall_score(y_test, pred,average='macro')
  f1 = f1_score(y_test, pred,average='macro')

  return (accuracy, precision, recall, f1) #튜플로 반환

In [ ]:
# 여러 가지 분류 모델을 한꺼번에 수행
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
# 모델
model_dic = {
    "Nearest Neighbors" : KNeighborsClassifier(3),
    "SVM" : SVC(),
    "Linear SVM" : SVC(kernel='linear', C=0.025),
    "RBF SVM" : SVC(gamma=2, C=1),
    "Decision Tree" : DecisionTreeClassifier(),
    "Random Forest" : RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    "LogisticRegression" : LogisticRegression(),
    "Adaboost" : AdaBoostClassifier(),
    "Naive Bayes" : GaussianNB(),
    "QDA" : QuadraticDiscriminantAnalysis()
}

In [ ]:
model_p = {}
model_acc = {}

for key in model_dic.keys() :
  model = model_dic[key]
  print(f'{key} : training')
  model.fit(X_train, y_train)

  print(f'{key} : predict')
  model_p[key] = model.predict(X_test)
  model_acc[key] = get_clf_eval(y_test, model_p[key])

Nearest Neighbors : training
Nearest Neighbors : predict
SVM : training
SVM : predict
Linear SVM : training
Linear SVM : predict
RBF SVM : training
RBF SVM : predict
Decision Tree : training
Decision Tree : predict
Random Forest : training
Random Forest : predict
LogisticRegression : training


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression : predict
Adaboost : training
Adaboost : predict
Naive Bayes : training
Naive Bayes : predict
QDA : training
QDA : predict


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model_acc
# 튜플로 반환

{'Nearest Neighbors': (0.963963963963964,
  0.9743589743589743,
  0.8707627118644068,
  0.901016401016401),
 'SVM': (0.7567567567567568,
  0.38035714285714284,
  0.4343825665859564,
  0.403968253968254),
 'Linear SVM': (0.972972972972973,
  0.9847222222222223,
  0.9503081664098614,
  0.9651763522310333),
 'RBF SVM': (0.963963963963964,
  0.9841269841269842,
  0.8678571428571429,
  0.9048467569493941),
 'Decision Tree': (0.972972972972973,
  0.9361111111111111,
  0.9095238095238095,
  0.9212548015364916),
 'Random Forest': (0.972972972972973,
  0.984450337512054,
  0.944047619047619,
  0.9622364953886693),
 'LogisticRegression': (0.9009009009009009,
  0.8253295303500221,
  0.7613306185340084,
  0.7872408293460924),
 'Adaboost': (0.6036036036036037,
  0.30760368663594473,
  0.3768765133171913,
  0.3269854906452845),
 'Naive Bayes': (0.6936936936936937,
  0.8367269984917044,
  0.5296885318071759,
  0.5662286302050081),
 'QDA': (0.990990990990991,
  0.9930555555555556,
  0.9957627118644068

In [ ]:
# 데이터프레임으로 변환
model_acc = pd.DataFrame.from_dict(model_acc,orient='index',columns= ['accuracy','precision','recall','f1'])

model_acc.sort_values('accuracy', ascending=False)

,accuracy,precision,recall,f1
QDA,0.990991,0.993056,0.995763,0.994342
Linear SVM,0.972973,0.984722,0.950308,0.965176
Decision Tree,0.972973,0.936111,0.909524,0.921255
Random Forest,0.972973,0.984450,0.944048,0.962236
Nearest Neighbors,0.963964,0.974359,0.870763,0.901016
RBF SVM,0.963964,0.984127,0.867857,0.904847
LogisticRegression,0.900901,0.825330,0.761331,0.787241
SVM,0.756757,0.380357,0.434383,0.403968
Naive Bayes,0.693694,0.836727,0.529689,0.566229
Adaboost,0.603604,0.307604,0.376877,0.326985


QDA가 가장 높게 나온다